In [37]:
%matplotlib inline
# Author: Samiran Dutta
# Date: 28.02.25

# This code replicates the model in 'Labor Market Power' by Berger et al. (2022)
# ..with new model visualisations (not in the original paper) + simple OLS estimation of how wage markdown affects wages from model generated data
# Research question: How do labor market imperfections affect wages? In particular, what are the origins and role of labor monopsony power in determining wages? 

## Packages

In [29]:
import numpy as np
from scipy.stats import pareto
import math

# Use JupyterDash for inline dashboards
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots
import plotly.graph_objects as go

print("Imports completed.")

Imports completed.


## Functions

### 1. Define Parameters

In [30]:
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Set parameters 
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def set_par(phi=0.50, alpha=0.987, beta=1/1.04, delta=0.1, eta=3.74, theta=0.76,
            J=1000, min_ij=1, max_ij=100, upsilon=0.20, gamma=0.818):
    R = (1/beta) - 1 + delta  # equilibrium capital steady‐state condition
    return {"phi": phi, "alpha": alpha, "beta": beta, "delta": delta,
            "eta": eta, "theta": theta, "R": R, "J": J,
            "min_ij": min_ij, "max_ij": max_ij, "upsilon": upsilon, "gamma": gamma}

### 2. Generate Number of Firms (for all i in each j = 1,...,J)

In [31]:
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Generate number of firms in each market 
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def gen_firms(params):
    J = params["J"]
    max_ij = params["max_ij"]
    Mj = np.zeros(J, dtype=int)
    prob_1_firm = 0.15  # 15% of markets have exactly 1 firm
    shape = 0.67
    scale = 5.7
    location = 2
    for j in range(J):
        if np.random.rand() < prob_1_firm:
            Mj[j] = 1
        else:
            # draw from a Pareto and shift by location; then take ceiling
            sample = math.ceil(pareto.rvs(shape, scale=scale) + location)
            Mj[j] = int(min(max_ij, max(1, sample)))
    Mj_max = Mj.max()
    return Mj, Mj_max

### 3. Generate Firm-level Productivity; drawn from a Pareto Distribution

In [32]:
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Generate firm-level productivities 
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def gen_productivity(params):
    J = params["J"]
    gamma = params["gamma"]
    alpha = params["alpha"]
    R = params["R"]
    Mj, Mj_max = gen_firms(params)
    
    # Initialize arrays (shape: (Mj_max, J))
    z_ij_notilde = np.zeros((Mj_max, J))
    z_ij = np.zeros((Mj_max, J))
    xi = 0.5  # standard deviation for the underlying normal

    # Use np.random.lognormal (mean and sigma are for the underlying normal)
    for j in range(J):
        m = Mj[j]
        if m > 0:
            z_ij_notilde[:m, j] = np.random.lognormal(mean=1, sigma=xi, size=m)
    
    # Compute the scaling factor and adjust productivity draws
    factor = (1 - (1 - gamma)*alpha) * ((alpha*(1-gamma))/R)**((alpha*(1-gamma))/(1 - alpha*(1-gamma)))
    z_ij = factor * np.power(z_ij_notilde, 1/(1 - alpha*(1-gamma)))
    
    return {"Mj": Mj, "Mj_max": Mj_max, "z_ij": z_ij, "z_ij_notilde": z_ij_notilde}

### 4. Steady-State Model (Wage-Bill Share Convergence) 

In [33]:
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Solve the model 
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def sect_eqlib(params):
    alpha = params["alpha"]
    beta = params["beta"]
    R = params["R"]
    eta = params["eta"]
    theta = params["theta"]
    J = params["J"]
    upsilon = params["upsilon"]
    phi = params["phi"]
    gamma = params["gamma"]
    
    prod = gen_productivity(params)
    Mj = prod["Mj"]
    Mj_max = prod["Mj_max"]
    z_ij = prod["z_ij"]
    z_ij_notilde = prod["z_ij_notilde"]
    
    # Initialize matrices for wages, markdown, shares, and labor elasticity.
    what_ij = np.zeros((Mj_max, J))
    mu_ij   = np.zeros((Mj_max, J))
    s_ij    = np.zeros((Mj_max, J))
    eps_ij  = np.zeros((Mj_max, J))
    
    max_iter = 1000
    alpha_tilde = 0.984
    
    for j in range(J):
        m = Mj[j]
        if m == 0:
            continue
        z_i = z_ij[:m, j]
        w_i = np.zeros(m)
        mu_i = np.zeros(m)
        eps_i = np.zeros(m)
        # initial guess for shares: proportional to productivity
        s_i = z_i / np.sum(z_i)
        iter_count = 0
        while iter_count <= max_iter:
            iter_count += 1
            eps_i = 1.0 / (s_i/theta + (1 - s_i)/eta)
            mu_i = eps_i / (eps_i + 1)
            a1 = 1.0 / (1 + (1 - alpha_tilde)*theta)
            a2 = -(1 - alpha_tilde)*(eta - theta) / (eta + 1)
            w_i = np.power(mu_i * alpha_tilde * z_i * np.power(s_i, a2), a1)
            W_j = np.power(np.sum(np.power(w_i, eta+1)), 1/(eta+1))
            s_i_new = np.power(w_i / W_j, eta+1)
            dist_S = np.max(np.abs(s_i_new - s_i))
            if dist_S < 1e-5:
                print(f"Market {j+1}: Shares converged in {iter_count} iterations! Max. difference = {dist_S:.2e}")
                break
            s_i = upsilon * s_i_new + (1 - upsilon) * s_i
            s_i = s_i / np.sum(s_i)
        # Store market-specific results into the full matrices
        what_ij[:m, j] = w_i
        mu_ij[:m, j]   = mu_i
        s_ij[:m, j]    = s_i
        eps_ij[:m, j]  = eps_i

    # Compute sectoral wage indices
    what_j = np.array([np.power(np.sum(np.power(what_ij[:Mj[j], j], eta+1)), 1/(eta+1)) for j in range(J)])
    What = np.power(np.sum(np.power(what_j, eta+1)), 1/(eta+1))
    
    # Compute nhat_ij with appropriate broadcasting
    nhat_ij = np.power(what_ij / what_j[None, :], eta) * np.power(what_j / What, theta)[None, :] * (What**phi)
    
    AveFirmSizehat_model = np.sum(nhat_ij) / np.sum(Mj)
    AveEarningshat_model = np.sum(what_ij * nhat_ij) / np.sum(nhat_ij)
    
    # Data targets
    AveFirmSize_data = 10
    AveEarnings_data = 65000
    
    phi_tilde = (AveFirmSize_data / AveFirmSizehat_model) / ((AveEarnings_data / AveEarningshat_model)**phi)
    Z_tilde = (phi_tilde**(1 - alpha_tilde)) * ((AveEarnings_data / AveEarningshat_model)**(1 + (1 - alpha_tilde)*phi)) \
              * (What** (-(1 - alpha_tilde)*(theta - phi)))
    
    # Scale up objects
    omega = Z_tilde / (phi_tilde**(1 - alpha))
    W = np.power(omega, 1/(1+(1-alpha)*phi)) * np.power(What, (1+(1-alpha)*theta)/(1+(1-alpha)*phi))
    w_ij = np.power(omega, 1/(1+(1-alpha)*theta)) * np.power(W, ((1-alpha)*(theta-phi))/(1+(1-alpha)*theta)) * what_ij
    w_j = np.array([np.power(np.sum(np.power(w_ij[:Mj[j], j], eta+1)), 1/(eta+1)) for j in range(J)])
    n_ij = phi_tilde * np.power(w_ij / w_j[None, :], eta) * np.power(w_j / W, theta)[None, :] * (W**phi)
    n_j = np.array([np.power(np.sum(np.power(n_ij[:Mj[j], j], (eta+1)/eta)), eta/(eta+1)) for j in range(J)])
    N = np.power(np.sum(np.power(n_j, (theta+1)/theta)), theta/(theta+1))
    lshare_ij = alpha_tilde * mu_ij
    Z_bar = Z_tilde**(1 - (1-gamma)*alpha)
    k_ij = np.power(((1 - gamma) * alpha * z_ij * Z_bar / R), 1/(1 - (1-gamma)*alpha)) \
           * np.power(n_ij, (gamma * alpha)/(1 - (1-gamma)*alpha))
    y_ij = Z_bar * z_ij_notilde * np.power(np.power(k_ij, 1-gamma) * np.power(n_ij, gamma), alpha)
    pi_ij = y_ij - w_ij * n_ij - R * k_ij

    AveFirmSizehat_model = np.sum(n_ij) / np.sum(Mj)
    AveEarningshat_model = np.sum(w_ij * n_ij) / np.sum(n_ij)
    print(f"Equilibrium computation completed for all {J} markets!")
    
    return {"Z_bar": Z_bar, "pi_ij": pi_ij, "z_ij": z_ij, "y_ij": y_ij, "w_ij": w_ij,
            "n_ij": n_ij, "k_ij": k_ij, "s_ij": s_ij, "mu_ij": mu_ij, "eps_ij": eps_ij,
            "lshare_ij": lshare_ij, "Mj": Mj, "N": N,
            "AveFirmSizehat_model": AveFirmSizehat_model, "AveEarningshat_model": AveEarningshat_model}

print("Function definitions loaded.")

Function definitions loaded.


## Run Simulation 

In [34]:
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Run the simulation 
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
params = set_par()
model = sect_eqlib(params)
z_ij = model["z_ij"]
Mj = model["Mj"]
w_ij = model["w_ij"]
s_ij = model["s_ij"]
eps_ij = model["eps_ij"]
mu_ij = model["mu_ij"]
n_ij = model["n_ij"]
k_ij = model["k_ij"]
y_ij = model["y_ij"]

print("Simulation completed.")

Market 1: Shares converged in 40 iterations! Max. difference = 8.85e-06
Market 2: Shares converged in 38 iterations! Max. difference = 8.86e-06
Market 3: Shares converged in 38 iterations! Max. difference = 9.87e-06
Market 4: Shares converged in 37 iterations! Max. difference = 8.71e-06
Market 5: Shares converged in 38 iterations! Max. difference = 8.99e-06
Market 6: Shares converged in 36 iterations! Max. difference = 8.57e-06
Market 7: Shares converged in 37 iterations! Max. difference = 8.65e-06
Market 8: Shares converged in 1 iterations! Max. difference = 0.00e+00
Market 9: Shares converged in 1 iterations! Max. difference = 0.00e+00
Market 10: Shares converged in 37 iterations! Max. difference = 9.32e-06
Market 11: Shares converged in 38 iterations! Max. difference = 8.09e-06
Market 12: Shares converged in 36 iterations! Max. difference = 9.59e-06
Market 13: Shares converged in 39 iterations! Max. difference = 9.17e-06
Market 14: Shares converged in 37 iterations! Max. difference 

## Plot Steady-State Objects

#### 1. Wages ($w_{ij}$), Wage-Bill Share ($s_{ij}$), Labor Supply Elasticity ($\epsilon_{ij}$) and Wage Markdown ($\mu_{ij}$) against percentiles of productivity ($z_{ij}$)

In [39]:
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Prepare data for plotting
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
eta = params["eta"]
theta = params["theta"]
mu_eta_ref = eta / (eta + 1)
mu_theta_ref = theta / (theta + 1)

J = params["J"]

# Find a market with exactly one firm and one with 10 active firms.
market_with_one_firm = next((j for j in range(J) if np.count_nonzero(z_ij[:, j] > 0) == 1), None)
market_with_multiple_firms = next((j for j in range(J) if np.count_nonzero(z_ij[:, j] > 0) == 10), None)

if market_with_one_firm is None or market_with_multiple_firms is None:
    raise ValueError("Required markets (one with exactly 1 firm and one with 10 firms) not found.")

selected_markets = [market_with_one_firm, market_with_multiple_firms]
market_labels = ["Single-firm market", "Multi-firm market"]
colors = ["blue", "red"]

percentile_labels = ["10th", "25th", "50th", "75th", "90th"]
x_positions = np.arange(1, len(percentile_labels)+1)

def compute_percentiles(z_values, y_values):
    # Compute quantile thresholds (10th, 25th, 50th, 75th, 90th)
    qs = np.percentile(z_values, [10, 25, 50, 75, 90])
    # For each quantile, find the index where z_value is closest to the quantile threshold
    indices = [np.argmin(np.abs(z_values - q)) for q in qs]
    y_percentiles = y_values[indices]
    return y_percentiles

print("Plotting data prepared.")

# -----------------------------
# Create a 2x2 subplot figure using Plotly
# -----------------------------
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=("Wages (w_ij)", "Wage-Bill Share (s_ij)",
                                    "Labor Elasticity (eps_ij)", "Markdown (mu_ij)"))

for idx, j in enumerate(selected_markets):
    m = Mj[j]
    if m == 1:
        xs = [x_positions[0]]
        s_vals = [s_ij[0, j]]
        w_vals = [w_ij[0, j]]
        eps_vals = [eps_ij[0, j]]
        mu_vals = [mu_ij[0, j]]
    else:
        z_values = z_ij[:m, j]
        xs = x_positions
        s_vals = compute_percentiles(z_values, s_ij[:m, j])
        w_vals = compute_percentiles(z_values, w_ij[:m, j])
        eps_vals = compute_percentiles(z_values, eps_ij[:m, j])
        mu_vals = compute_percentiles(z_values, mu_ij[:m, j])
    
    fig.add_trace(go.Scatter(x=xs, y=w_vals, mode='lines+markers',
                             name=market_labels[idx], marker_color=colors[idx],
                             showlegend=(idx==0)), row=1, col=1)
    fig.add_trace(go.Scatter(x=xs, y=s_vals, mode='lines+markers',
                             name=market_labels[idx], marker_color=colors[idx],
                             showlegend=False), row=1, col=2)
    fig.add_trace(go.Scatter(x=xs, y=eps_vals, mode='lines+markers',
                             name=market_labels[idx], marker_color=colors[idx],
                             showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=xs, y=mu_vals, mode='lines+markers',
                             name=market_labels[idx], marker_color=colors[idx],
                             showlegend=False), row=2, col=2)

# Add horizontal dashed lines to the markdown subplot.
fig.add_hline(y=mu_eta_ref, line=dict(dash="dash", color="black"),
              row=2, col=2, annotation_text="η/(η+1)", annotation_position="bottom left")
fig.add_hline(y=mu_theta_ref, line=dict(dash="dash", color="gray"),
              row=2, col=2, annotation_text="θ/(θ+1)", annotation_position="bottom right")

fig.update_xaxes(title_text="Productivity Percentiles", tickvals=list(x_positions),
                 ticktext=percentile_labels)
fig.update_yaxes(title_text="Wages", row=1, col=1)
fig.update_yaxes(title_text="Wage-Bill Share", row=1, col=2)
fig.update_yaxes(title_text="Labor Elasticity", row=2, col=1)
fig.update_yaxes(title_text="Markdown", row=2, col=2)

fig.update_layout(height=800, width=1000, title_text="Sectoral Equilibrium Metrics by Productivity Percentile")

fig.show()

Plotting data prepared.


#### Relationship Between Productivity, Wages, and Labor Market Power

- More productive firms pay higher wages and capture a larger share of wage bills in their markets.
- A larger wage-bill share reduces labor supply elasticity to the firm, increasing its market presence.
- Lower labor supply elasticity enhances monopsony power, leading to lower markdowns i.e., higher labor market power since $w_{ij} = \mu_{ij} MRPL_{ij}$

- Firms operating in single-market environments exhibit the highest monopsony power, consistent with classic models.


#### 2. Firm-Size Distribution 

In [38]:
# -----------------------------
# Create histogram of employment (n_ij)
# -----------------------------
active_n_ij = n_ij[n_ij > 0].flatten()
hist_fig = go.Figure(data=[go.Histogram(x=active_n_ij, nbinsx=100)])
hist_fig.update_layout(title_text="Firm-Size Distribution",
                         xaxis_title="Employment (n_ij)",
                         yaxis_title="Frequency")
hist_fig.show()

The model also generates a realistic firm-size distribution, where the average firm-size can be tweaked to match the data.

## Regression

#### A simple OLS regression: $w_{ij} = \beta_0 + \beta_1 \mu_{ij} + \epsilon_{ij}$

In [26]:

import numpy as np
import statsmodels.api as sm

# Extract w_ij and mu_ij from the model dictionary
w_ij = model["w_ij"].flatten()  # Flatten to 1D array
mu_ij = model["mu_ij"].flatten()  # Flatten to 1D array

# Remove NaN or infinite values if any
valid_indices = np.isfinite(w_ij) & np.isfinite(mu_ij)
w_ij_clean = w_ij[valid_indices]
mu_ij_clean = mu_ij[valid_indices]

# Add a constant term for the regression (intercept)
mu_ij_clean = sm.add_constant(mu_ij_clean)

# Run the regression
regression_model = sm.OLS(w_ij_clean, mu_ij_clean).fit()

# Display the regression results
print(regression_model.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.656
Model:                            OLS   Adj. R-squared:                  0.656
Method:                 Least Squares   F-statistic:                 1.911e+05
Date:                Fri, 28 Feb 2025   Prob (F-statistic):               0.00
Time:                        12:29:53   Log-Likelihood:            -1.0780e+06
No. Observations:              100000   AIC:                         2.156e+06
Df Residuals:                   99998   BIC:                         2.156e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        284.3778     43.896      6.478      0.0

This reveals a positive correlation, as expected. Thus, labor monopsony power (inverse of $\mu_{ij}$) originating from higher productivity (structually from the model) leads to a decline in wages.